# ECoG Foundation Model Training
This is meant to be a minimal notebook which is capable of running model training with a free to use colab notebooks. Feel free to change this as you see fit for your experiments.

In [ ]:
# Clone repository.
!git clone https://github.com/leoniekerken/ECoG-foundation-model.git

Now, go into the repo you just downloaded and change the hugging face user access token in the Makefile to your personal access token. If you don't want to do this everytime you could also upload the code to your personal drive and change the path_to_github_repo variable below, although then you risk your code being out of date.

In [ ]:
# Download data.
!cd ECoG-foundation-model && make download-data

In [ ]:
# The local path to the github repo. Must be accessible from this notebook.
# If you just run the code above this will work.
path_to_github_repo = 'ECoG-foundation-model/'

In [ ]:
# Required pip installs.
!pip install accelerate
!pip install einops
!pip install mne
!pip install mne-bids
!pip install pyEDFlib

In [ ]:
# Add import for ECoG code.
import sys
import os
sys.path.append(os.path.join(path_to_github_repo, 'ECoG-MAE'))

# Other imports
from dataclasses import dataclass
from config import system_setup, model_setup
from loader import dl_setup
from train import train_model

In [ ]:
# Set up args for training.
@dataclass
class ArgObject:
    # If 'batch' then will normalize data within a batch. If
    norm: str = None
    data_size: float = 0.0
    # If true then convert data to power envelope by taking magnitude of Hilbert
    # transform.
    env: bool = False
    # Frequency bands for filtering raw iEEG data.
    bands: list[list[int]] = None
    # Frequency to resample data to.
    new_fs: int = 0
    # Batch size to train with.
    batch_size: int = 32
    # Relative path to the dataset root directory.
    dataset_path: str = None
    # Proportion of data to have in training set. The rest will go to test set.
    train_data_proportion: float = 0.9
    # If true then use contrastive loss to train model. Currently not supported.
    use_contrastive_loss: bool = False
    # Prepend classification token to input if True. Always True if
    # use_contrastive_loss is True.
    use_cls_token: bool = False
    # Number of seconds of data to use for a training example.
    sample_length: int = 0
    # TODO
    patch_size: list[int] = None
    # TODO
    frame_patch_size: int = 0
    # Proportion of tubes to mask out. See VideoMAE paper for details.
    tube_mask_ratio: float = 0.5
    # Proportion of
    decoder_mask_ratio: float = 0
    # Dimensionality of token embeddings.
    dim: int = 512
    # Dimensionality of feedforward network after attention layer.
    mlp_dim: int = 512
    # Learning rate for training. If 0 then uses Adam scheduler.
    learning_rate: float = 0.0
    # Number of epochs to train over data.
    num_epochs: int = 0
    # Name of training job. Will be used to save metrics.
    job_name: str = None


# TODO: test batch size of 64 to see if VRAM runs out.
args_dict = {
    'norm': 'batch',
    'data_size': 1.0,
    'env': True,
    'new_fs': 20,
    # Currently set to 9 because get_model_recon.py in metrics.py assumes that
    # the batch size is at least 9.
    # See https://github.com/leoniekerken/ECoG-foundation-model/issues/1 about
    # maybe changing that. I'm not sure on the exact limits but I've managed to
    # get a batch size of 32 to work but a batch size of 64 leads to crashes on
    # the free tier T4 GPU.
    'batch_size': 32,
    'bands': [[4,8],[8,13],[13,30],[30,55],[70,200]],
    'sample_length': 2,
    'patch_size': [1, 2, 2],
    'frame_patch_size': 4,
    'tube_mask_ratio': 0.5,
    'decoder_mask_ratio': 0.0,
    'dim': 80,
    'mlp_dim': 80,
    'learning_rate': 0.0,
    'num_epochs': 10,
    'job_name': 'test_overfitting',
    'dataset_path': 'ECoG-foundation-model/dataset',
    'train_data_proportion': 0.5,
}
args = ArgObject(**args_dict)

In [ ]:
accelerator, device, data_type, local_rank = system_setup()

In [ ]:
train_dl, test_dl, num_train_samples = dl_setup(args)

# If you want to run a more minimal training run you can uncomment the code
# below to limit the number of samples accessible by each dataset. This is
# currently inefficient though if you use few training batches because it will
# do summaries frequently which takes more time than an individual training
# step.
# train_num_batches = 1
# test_num_batches = 1

# for dataset in train_dl.dataset.datasets:
#   dataset.max_samples = args.batch_size * train_num_batches
# # Can reuse same dataloader for test
# test_dl = train_dl
# # Or just limit test dataloader
# # for dataset in test_dl.dataset.datasets:
# #   dataset.max_samples = args.batch_size * test_num_batches

# num_train_samples = args.batch_size * train_num_batches

In [ ]:
# The data is arranged in shape b*c*t*d*h*w, where
# b = batch size,
# c = freq bands,
# t = number of datapoints within a sample (args.new_fs samples per second)
# d = depth (currently 1)
# h = height of grid (currently 8)
# w = width of grid (currently 8)

print(next(train_dl._get_iterator())['signal'].shape)

In [ ]:
model, optimizer, lr_scheduler, num_patches = model_setup(
    args, device, num_train_samples
)

In [ ]:
model = train_model(
        args,
        device,
        model,
        train_dl,
        test_dl,
        num_patches,
        optimizer,
        lr_scheduler,
        accelerator,
        data_type,
        local_rank,
    )

You can now view the results of the training in results/